In [ ]:
import torch

# Check if CUDA is available
# it is not requires to use cuda.
torch.cuda.is_available()

Bevore you run the script download the following papers and store it in `langchain-workshop/llm-papers`. You can also skip the downloading by using one the cell I mentioned below.

[dpo](https://arxiv.org/abs/2305.18290)

[self-rag](https://arxiv.org/abs/2310.11511)

[medagents](https://arxiv.org/abs/2311.10537)

[raptor](https://arxiv.org/abs/2401.18059)

[replacing judges with juries](https://arxiv.org/abs/2404.18796)

[IPO](https://arxiv.org/abs/2404.19733)

[lora land](https://arxiv.org/abs/2405.00732)



```python
from langchain_community.document_loaders import ArxivLoader


arxiv_ids = ["2305.18290","2310.11511","2311.10537","2401.18059","2404.18796,"2404.19733","2405.00732"]

docs = []
for arxiv_id in arxiv_ids:
    doc = ArxivLoader(query="2305.18290", load_max_docs=1).load()[0]
    doc.metadata = {"metadata":doc.metdata}
    
```


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
from transformers import (
    AutoTokenizer,
)
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Qdrant
import platform


def check_os() -> str:
    """Check the OS of the system"""
    os_name = platform.system()
    return os_name
# loading the embedding model
embedding_path = Path("huggingface_models","BAAI","bge-large-en-v1.5")
embeddings = HuggingFaceEmbeddings(model_name=embedding_path)
emb_tokenizer = AutoTokenizer.from_pretrained(embedding_path)
def _measure_length_with_tokenizer(text: str) -> int:
    """Measure the length of the text with tokenizer.

    Args:
        text: the text to be measured.

    Returns:
        The number of tokens of the input text.
    """
    return len(emb_tokenizer.encode(text))
# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    length_function=_measure_length_with_tokenizer,
    separators=["\n\n", "\n", ".", "?", ":", ",", ":", "!", " ", ""],
)


You can replace the cell below with 


```python
from langchain_community.document_loaders import ArxivLoader


arxiv_ids = ["2305.18290","2310.11511","2311.10537","2401.18059","2404.18796,"2404.19733","2405.00732"]

docs = []
for arxiv_id in arxiv_ids:
    doc = ArxivLoader(query="2305.18290", load_max_docs=1).load()[0]
    doc.metadata = {"metadata":doc.metdata}
    
```


In [ ]:
docs = []
paper_paths = list(Path("llm-papers").glob("*.pdf"))
docs = []
if check_os() == "Windows":
    title_splitter = "\\"
else:
    title_splitter = "/"
for paper in paper_paths:
    paper_pages = PyPDFLoader(str(paper)).load()
    title = paper_pages[0].metadata["source"]
    normalized_title = " ".join(
        title.split(title_splitter)[-1][:-4].replace("_", " ").split(".")[-1].split()
    )
    content = ""
    for page in paper_pages:
        content += page.page_content + "\n"
    paper_pages[0].page_content = content
    paper_pages[0].metadata = {"source": normalized_title}
    docs.append(paper_pages[0])

# splitting the document into chunks

In [ ]:
splits = text_splitter.split_documents(docs)
chunk_index = 0
current_title = ""
for split in splits:
    if current_title != split.metadata["source"]:
        chunk_index = 0
        current_title = split.metadata["source"]
    split.metadata["chunk"] = chunk_index
    split.metadata = {"metadata": split.metadata}
    chunk_index += 1

# Create a local qdrant database

In [ ]:
qdrant = Qdrant.from_documents(
    splits,
    embeddings,
    path="./qdrant-database",
    collection_name="llm_papers",
)

In [ ]:
qdrant.search("self-rag", search_type="similarity", k=5)